In [1]:
#Buliding the Neural network model
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

2023-05-04 16:52:48.870710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#the path to images
path_to_images = 'Data/dataset1/known_images'


In [3]:
# Training and validating the dataset
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(path_to_images, class_names = ['Normal','Sick'], batch_size = 16, image_size = (395, 488), validation_split = 0.2, subset = "both", seed = 200)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


2023-05-04 16:52:51.666878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Creating the input shape
inputs = keras.Input(shape = (395,488,3))

#creating a convolutional layer with max pooling
X = keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')(inputs)
X = keras.layers.MaxPooling2D(pool_size = 2)(X)

X = keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')(inputs)
X = keras.layers.MaxPooling2D(pool_size = 2)(X)

X = keras.layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu')(inputs)
X = keras.layers.MaxPooling2D(pool_size = 2)(X)

X = keras.layers.Conv2D(filters = 256, kernel_size = 3, activation = 'relu')(inputs)
X = keras.layers.MaxPooling2D(pool_size = 2)(X)

X = keras.layers.Conv2D(filters = 256, kernel_size = 3, activation = 'relu')(inputs)
X = keras.layers.MaxPooling2D(pool_size = 2)(X)

X = keras.layers.Flatten()(X)

#X = keras.layers.Dense(64, activation = 'relu')(X)

#Creating the model output
outputs = keras.layers.Dense(1, activation = 'sigmoid')(X)



In [5]:
model = keras.Model(inputs = inputs, outputs = outputs)

In [6]:
#Compiling the model

model.compile(
loss = 'binary_crossentropy',
optimizer = 'rmsprop',
metrics = ('accuracy')

)

In [ ]:

history = model.fit(
train_dataset,
epochs = 10,
validation_data = val_dataset
)

Epoch 1/10
36/36 [==============================] - 339s 9s/step - loss: 1982.8849 - accuracy: 0.7710 - val_loss: 11.2972 - val_accuracy: 0.9441
Epoch 2/10
 9/36 [======>.......................] - ETA: 3:38 - loss: 10.6531 - accuracy: 0.9444

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history ['accuracy']
val_accuracy = history.history ['val_accuracy']
loss = history.history ['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label = 'Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss,'bo', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.legend
plt.show()
